In [1]:
# Import pandas library using an alias
import pandas as pd
# library to handle data in a vectorized manner
import numpy as np

# library to handle JSON files
# import json
# library to handle requests
# import requests
# tranform JSON file into a pandas dataframe
# from pandas.io.json import json_normalize

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

import plotly.express as px

In [2]:
#library_df = pd.read_excel('ChicagoLibraries.xlsx')
#library_df.head()
ward49_df = pd.read_excel('Ward49.xlsx')
ward49_df.head()

,name,address,city,state,zip,phone,hours_of_operation,latitude,longitude,url,location_type
0,Rogers Park,6907 N. Clark St.,Chicago,IL,60626.0,(312) 744-0156,"Sun., Closed; Mon. & Wed., Noon-8; Tue. & Thu....",42.006708,-87.673408,https://www.chipublib.org/locations/61/,Library
1,SULLIVAN HS,6631 N BOSWORTH AVE,NaN,NaN,NaN,NaN,NaN,42.002688,-87.669192,NaN,High School
2,GALE,1631 W JONQUIL TER,NaN,NaN,NaN,NaN,NaN,42.021091,-87.671939,NaN,Elementary School
3,ACERO - DE LA CRUZ,7416 N RIDGE BLVD,NaN,NaN,NaN,NaN,NaN,42.016476,-87.684406,NaN,Elementary School
4,FIELD,7019 N ASHLAND AVE,NaN,NaN,NaN,NaN,NaN,42.009664,-87.669985,NaN,Elementary School


In [3]:
#library_df.columns
ward49_df.columns

Index(['name', 'address', 'city', 'state', 'zip', 'phone',
       'hours_of_operation', 'latitude', 'longitude', 'url', 'location_type'],
      dtype='object')

In [4]:
# get list of location types
location_type_list = ward49_df['location_type'].drop_duplicates().to_list()
location_type_list

['Library', 'High School', 'Elementary School']

In [5]:
# Get coordinates for Chicago
#address = 'Chicago, IL'
address = 'Rogers Park, Chicago, IL'
geolocator = Nominatim(user_agent='chi_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geographical coordinates of Rogers Park, Chicago, IL are {}, {}".format(latitude, longitude))

The geographical coordinates of Rogers Park, Chicago, IL are 42.0090367, -87.6768492


## Create basic map

In [18]:
# Create a map
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(ward49_df,
                        lat="latitude",
                        lon="longitude",
                        hover_data=["name","address"],
                        zoom=12)
#fig.update_layout(mapbox_style="open-street-map")
#fig.update_layout(mapbox_style="basic")
fig.update_layout(mapbox_style="streets")
fig.show()

In [42]:
# Add markers to the map
for lat, lon, name in zip(ward49_df['latitude'],
                          ward49_df['longitude'],
                          ward49_df['name']):
    label = folium.Popup(str(name), parse_html=True)
    folium.CircleMarker(
       [lat, lon],
       radius=5,
       popup=label,
       fill=True,
       fill_opacity=0.7).add_to(chicago_map)

In [43]:
# Display the map
chicago_map

## Test interactive map

In [44]:
# Create a map
chicago_map = folium.Map(location=[latitude, longitude],
                         zoom_start=14)

In [45]:
#ward49_df[ward49_df['location_type'] == 'Library']
ward49_df[ward49_df['location_type'] == 'Library'][['latitude','longitude']]

,latitude,longitude
0,42.006708,-87.673408


In [46]:
points1 = [(42.006708, -87.673408)]

library_group = folium.FeatureGroup(name='Library').add_to(chicago_map)

for tuple_ in points1:
    label = folium.Popup('Rogers Park Library', parse_html=True)
    library_group.add_child(folium.CircleMarker(
        (42.006708, -87.673408),
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7)
    )

In [47]:
ward49_df[ward49_df['location_type'] == 'High School'][['latitude','longitude']]

,latitude,longitude
1,42.002688,-87.669192
6,42.013031,-87.674818


In [48]:
points2 = [(42.002688, -87.669192),
           (42.013031, -87.674818)]

hs_group = folium.FeatureGroup(name='High School').add_to(chicago_map)

for tuple_ in points2:
    label = folium.Popup('High School', parse_html=True)
    hs_group.add_child(folium.CircleMarker(
        tuple_,
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7)
    )

In [49]:
folium.LayerControl().add_to(chicago_map)
chicago_map

## Create Interactive Map

In [50]:
# Create a map
chicago_map = folium.Map(location=[latitude, longitude],
                         zoom_start=14)

In [51]:
fgroup = []

In [52]:
for i in range(len(location_type_list)):
    #print(i, location_type_list[i])
    fgroup.append(folium.FeatureGroup(name=location_type_list[i]).add_to(chicago_map))
    for row in ward49_df[ward49_df['location_type'] == location_type_list[i]].itertuples():
        #print(row.name, row.latitude, row.longitude)
        label = folium.Popup(row.name, parse_html=True)
        fgroup[i].add_child(folium.CircleMarker(
            [row.latitude, row.longitude],
            radius=5,
            popup=label,
            fill=True,
            fill_opacity=0.7).add_to(chicago_map)
        )
    
fgroup

In [53]:
folium.LayerControl().add_to(chicago_map)
chicago_map

## Try different marker types

In [54]:
# Create a map
chicago_map = folium.Map(location=[latitude, longitude],
                         zoom_start=14)

In [57]:
# Add markers to the map
for lat, lon, name in zip(ward49_df['latitude'],
                          ward49_df['longitude'],
                          ward49_df['name']):
    label = folium.Popup(str(name), parse_html=True)
    icon = folium.Icon(color='red', icon='bell', prefix='fa')
    folium.Marker(
       location=[lat, lon],
       icon=icon,
       popup=label).add_to(chicago_map)
    
chicago_map

## Customize interactive map

In [64]:
# Create a map
chicago_map = folium.Map(location=[latitude, longitude],
                         zoom_start=13)

fgroup = []
for i in range(len(location_type_list)):
    #print(i, location_type_list[i])
    fgroup.append(folium.FeatureGroup(name=location_type_list[i]).add_to(chicago_map))
    
    for row in ward49_df[ward49_df['location_type'] == location_type_list[i]].itertuples():
        #print(row.name, row.latitude, row.longitude)
        label = folium.Popup(row.name, parse_html=True)
        if row.location_type == 'Elementary School':
            icon = folium.Icon(color='red', icon='bell', prefix='fa')
        elif row.location_type == 'High School':
            icon = folium.Icon(color='blue', icon='graduation-cap', prefix='fa')
        elif row.location_type == 'Library':
            icon = folium.Icon(color='green', icon='book', prefix='fa')
        else:
            icon = folium.Icon(color='gray', icon='question-circle', prefix='fa')

        fgroup[i].add_child(folium.Marker(
            location=[row.latitude, row.longitude],
            icon=icon,
            popup=label).add_to(chicago_map)
        )
        
folium.LayerControl().add_to(chicago_map)
chicago_map